The goal is to : 

1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:
    a.	 If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
    b.	 Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
    c.	 To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected.


Import the libraries.

In [702]:
from pulp import *
import pandas as pd

Load the diet data.

In [703]:
#df = pd.read_excel('data/diet.xls')

df = pd.read_excel(
        open(
            'data/diet.xls',
            'rb'
                ),
                sheet_name='Sheet1'
                )

In [704]:
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [705]:
diet_df = df[0:64]


Explore our diet data set.

In [706]:
diet_df.dtypes  # Variables data types.


Foods               object
Price/ Serving     float64
Serving Size        object
Calories           float64
Cholesterol mg     float64
Total_Fat g        float64
Sodium mg          float64
Carbohydrates g    float64
Dietary_Fiber g    float64
Protein g          float64
Vit_A IU           float64
Vit_C IU           float64
Calcium mg         float64
Iron mg            float64
dtype: object

In [707]:
diet_df.describe() # Return statistical analysis.


,Price/ Serving,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,0.327188,134.384375,18.071875,5.185937,332.126563,16.437500,1.503125,6.071875,777.132812,11.498438,50.296875,1.587500
std,0.254536,120.790475,42.114117,9.684530,491.718977,18.257971,2.216817,6.706771,2173.534758,28.957448,75.257796,2.477998
min,0.020000,2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900000,0.000000
25%,0.145000,73.400000,0.000000,0.500000,16.175000,4.625000,0.000000,1.975000,11.750000,0.000000,6.775000,0.300000
50%,0.270000,109.400000,0.000000,2.650000,137.250000,15.050000,0.700000,4.050000,95.450000,1.550000,20.400000,0.700000
75%,0.460000,159.500000,17.625000,6.700000,335.100000,22.150000,1.850000,8.025000,499.900000,7.525000,51.650000,2.025000
max,0.990000,710.800000,211.500000,72.200000,1915.100000,128.200000,11.600000,42.200000,15471.000000,160.200000,302.300000,16.800000


Let's check for missing data. We don't have any missing data.

In [708]:
diet_df.isnull().values.any()


False

Create lists for min and max.

In [709]:
a_min = [1500, 30, 20, 800, 130, 125, 60, 1000, 400, 700, 10]

In [710]:
a_max = [2500, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]

Convert our diet data frame into list of list

In [711]:
data = diet_df.values.tolist()


Make a list of foods

In [712]:
foods = [f[0] for f in data]


Create dictionaries for the various food properties/components. This will help if we want to optimize for specific food components.

In [713]:
calories = dict([(c[0], float(c[3])) for c in data])

In [714]:
cholesterol = dict([(c[0], float(c[4])) for c in data])

In [715]:
total_Fat = dict([(t[0], float(t[5])) for t in data])

In [716]:
sodium = dict([(s[0], float(s[6])) for s in data])

In [717]:
Carbohydrates = dict([(c[0], float(c[7])) for c in data])

In [718]:
dietary_Fiber = dict([(d[0], float(d[8])) for d in data])

In [719]:
protein = dict([(p[0], float(p[9])) for p in data])

In [720]:
vit_A = dict([(v[0], float(v[10])) for v in data])

In [721]:
vit_C = dict([(v[0], float(v[11])) for v in data])

In [722]:
calcium = dict([(c[0], float(c[12])) for c in data])

In [723]:
iron = dict([(i[0], float(i[13])) for i in data])


Define the cost dictionary.

In [724]:
cost = dict([(c[0], float(c[1])) for c in data])


Let's work on the contraints per variable by iterating through all of them and appending the contraint.

In [740]:
b = []
for j in range(0,11):
    b.append(dict([(x[0], float(x[j+3])) for x in data]))
 

Let's create an minimization optimization problem.

In [726]:
optimization_problem = LpProblem('DietProblem', LpMinimize)


Let's define binary variables. Whether the food is chosen or not representing 0 for not chosen and 1 for chosen.

In [727]:
chosen_variables = LpVariable.dicts("Chosen",foods,0,1,"Binary")
#chosen_variables = LpVariable.dicts("Chosen",foods,cat="Binary")


Let's define continuous variables.

In [728]:
food_variables = LpVariable.dicts("Foods", foods,0)


Let's define the objective function. We are going to use lpSum() function.

In [729]:
optimization_problem += lpSum([cost[f] * food_variables[f] for f in foods]),'total cost'


To solve : "If a food is selected, then a minimum of 1/10 serving must be chosen". We need to add these contraints. 
We set:- 
Upper bound(we multiplying by 1000000 - large number) 
and 
Lower bound(we multiplying by 0.1 - 1/10).

In [730]:
for f in foods:
    optimization_problem += food_variables[f] <= 1000000 * chosen_variables[f]
    optimization_problem += food_variables[f] >= 0.1 * chosen_variables[f]
    

Let's add the minimum daily intake contraints on all foods using our min list.

In [731]:
# Using min list.
for i in range(0,11):
    b_x_dot = lpSum([b[i][j] * food_variables[j] for j in foods])
    contraint_1 = a_min[i] <= + b_x_dot
    optimization_problem += contraint_1
    

Let's add the maximum daily intake contraints on all foods using our max list.

In [732]:
# Using max list.
for i in range(0,11):
    b_x_dot = lpSum([b[i][j] * food_variables[j] for j in foods])
    contraint_2 = a_max[i] >= + b_x_dot
    optimization_problem += contraint_2
    

To solve : "Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected". We need to add this contraint. 

In [733]:
optimization_problem += chosen_variables['Frozen Broccoli'] + \
chosen_variables['Celery, Raw'] <= 1, 'At most one Broccoli / Celery'


To solve : "To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected". We need to add this contraint.

In [734]:
optimization_problem += chosen_variables['Roasted Chicken'] + chosen_variables['Poached Eggs'] + \
  chosen_variables['Scrambled Eggs'] + chosen_variables['Frankfurter, Beef'] + \
  chosen_variables['Kielbasa,Prk'] + chosen_variables['Hamburger W/Toppings'] + \
  chosen_variables['Hotdog, Plain'] + chosen_variables['Pork'] + \
  chosen_variables['Bologna,Turkey'] + chosen_variables['Ham,Sliced,Extralean'] + \
  chosen_variables['White Tuna in Water'] \
  >= 3, 'At least three proteins'


Let's solve our diet optimization problem.

In [735]:
optimization_problem.solve()


1

In [736]:
VariablesDict = {} # A dict to store the optimal diet requirement.
print('The Optimal Solution:')
for var in optimization_problem.variables():
    if var.varValue > 0:
        if str(var).find('Chosen'):
            VariablesDict[var] = var.varValue
            print(str(var.varValue) + " units of " + str(var))
            

The Optimal Solution:
42.399358 units of Foods_Celery,_Raw
0.1 units of Foods_Kielbasa,Prk
82.802586 units of Foods_Lettuce,Iceberg,Raw
3.0771841 units of Foods_Oranges
1.9429716 units of Foods_Peanut_Butter
0.1 units of Foods_Poached_Eggs
13.223294 units of Foods_Popcorn,Air_Popped
0.1 units of Foods_Scrambled_Eggs


In [737]:
print(VariablesDict)


{Foods_Celery,_Raw: 42.399358, Foods_Kielbasa,Prk: 0.1, Foods_Lettuce,Iceberg,Raw: 82.802586, Foods_Oranges: 3.0771841, Foods_Peanut_Butter: 1.9429716, Foods_Poached_Eggs: 0.1, Foods_Popcorn,Air_Popped: 13.223294, Foods_Scrambled_Eggs: 0.1}


The total cost of food for the optimal diet requirement.    

In [738]:
print("Total cost of food = $%.2f" % value(optimization_problem.objective))


Total cost of food = $4.51
